In [ ]:
pip install qiskit qiskit-aer torch torch_geometric networkx matplotlib scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
import numpy as np
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv, TransformerConv
from torch_geometric.utils import to_undirected, negative_sampling
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
from scipy.constants import h, c
from sklearn.model_selection import KFold
import json
from datetime import datetime
import os
import torch.nn.functional as F

def bce_with_logits_loss(pos_out, neg_out):
    """Custom BCE loss for link prediction"""
    pos_loss = F.binary_cross_entropy_with_logits(
        pos_out, torch.ones_like(pos_out)
    )
    neg_loss = F.binary_cross_entropy_with_logits(
        neg_out, torch.zeros_like(neg_out)
    )
    return pos_loss + neg_loss

def to_networkx(data):
    """Convert PyG data to NetworkX graph"""
    G = nx.Graph()
    edge_index = data.edge_index.cpu().numpy()
    for i in range(edge_index.shape[1]):
        G.add_edge(edge_index[0, i], edge_index[1, i])
    return G

class AdvancedQuantumChannelSimulator:
    def __init__(self, distance, wavelength=1550e-9, fiber_loss=0.2,
                 detector_efficiency=0.1, dark_count_rate=1e-6,
                 atmospheric_visibility=None):
        self.distance = distance
        self.wavelength = wavelength
        self.fiber_loss = fiber_loss
        self.detector_efficiency = detector_efficiency
        self.dark_count_rate = dark_count_rate
        self.atmospheric_visibility = atmospheric_visibility
        self.photon_energy = h * c / wavelength

    def calculate_channel_loss(self):
        fiber_loss_db = self.fiber_loss * self.distance
        fiber_transmission = 10 ** (-fiber_loss_db/10)

        if self.atmospheric_visibility:
            beam_divergence = 1.22 * self.wavelength / 0.1
            geometric_loss = (0.1 / (beam_divergence * self.distance)) ** 2
            atmospheric_loss = np.exp(-3.91 * self.distance / self.atmospheric_visibility)
            total_transmission = fiber_transmission * geometric_loss * atmospheric_loss
        else:
            total_transmission = fiber_transmission

        return total_transmission

    def simulate_bb84_protocol(self, num_pulses=10000, mean_photon_number=0.1):
        channel_transmission = self.calculate_channel_loss()
        received_photons = np.random.poisson(
            mean_photon_number * channel_transmission * self.detector_efficiency,
            num_pulses
        )
        dark_counts = np.random.poisson(self.dark_count_rate, num_pulses)
        total_counts = received_photons + dark_counts
        basis_matches = np.random.choice([0, 1], num_pulses, p=[0.5, 0.5])
        qber = 0.5 * (1 - np.exp(-2 * self.distance / 100))
        errors = np.random.choice([0, 1], num_pulses, p=[1-qber, qber])
        matched_pulses = total_counts * basis_matches
        raw_key_rate = np.sum(matched_pulses) / num_pulses
        final_key_rate = raw_key_rate * (1 - 2 * h2(qber))

        return {
            'qber': qber,
            'raw_key_rate': raw_key_rate,
            'final_key_rate': final_key_rate,
            'channel_loss_db': -10 * np.log10(channel_transmission),
            'dark_count_probability': np.mean(dark_counts > 0)
        }

def h2(x):
    """Binary entropy function"""
    return -x * np.log2(x) - (1-x) * np.log2(1-x) if 0 < x < 1 else 0

class AdvancedQKDNetwork:
    def __init__(self, num_nodes=50):
        self.num_nodes = num_nodes
        self.positions = self._generate_realistic_topology()

    def _generate_realistic_topology(self):
        centers = np.random.multivariate_normal(
            mean=[0, 0],
            cov=[[100, 0], [0, 100]],
            size=3
        )

        positions = []
        for _ in range(self.num_nodes):
            center = centers[np.random.randint(0, 3)]
            pos = center + np.random.multivariate_normal(
                mean=[0, 0],
                cov=[[10, 0], [0, 10]]
            )
            positions.append(pos)

        return np.array(positions)

    def generate_graph_data(self):
        distances = np.zeros((self.num_nodes, self.num_nodes))
        for i in range(self.num_nodes):
            for j in range(i + 1, self.num_nodes):
                distances[i, j] = distances[j, i] = np.linalg.norm(
                    self.positions[i] - self.positions[j]
                )

        edges = []
        edge_attrs = []

        for i in range(self.num_nodes):
            for j in range(i + 1, self.num_nodes):
                if distances[i, j] < 100:
                    simulator = AdvancedQuantumChannelSimulator(
                        distance=distances[i, j],
                        atmospheric_visibility=20000 if np.random.random() < 0.2 else None
                    )
                    results = simulator.simulate_bb84_protocol()

                    if results['final_key_rate'] > 0:
                        edges.append([i, j])
                        edge_attrs.append([
                            results['final_key_rate'],
                            results['qber'],
                            distances[i, j],
                            results['channel_loss_db'],
                            results['dark_count_probability']
                        ])

        edge_index = torch.tensor(edges).t().contiguous()
        edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

        G = nx.Graph()
        G.add_edges_from(edges)

        node_features = []
        for i in range(self.num_nodes):
            features = [
                self.positions[i, 0],
                self.positions[i, 1],
                G.degree(i) if i in G else 0,
                nx.betweenness_centrality(G).get(i, 0) if i in G else 0
            ]
            node_features.append(features)

        return Data(
            x=torch.tensor(node_features, dtype=torch.float),
            edge_index=to_undirected(edge_index),
            edge_attr=edge_attr,
            pos=torch.tensor(self.positions, dtype=torch.float)
        )

class AdvancedQKDLinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, edge_attr_channels, hidden_channels=64):
        super().__init__()

        self.conv1 = TransformerConv(in_channels, hidden_channels)
        self.conv2 = GATv2Conv(hidden_channels, hidden_channels)

        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(edge_attr_channels, hidden_channels),
            torch.nn.LayerNorm(hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(hidden_channels, hidden_channels)
        )

        self.link_predictor = torch.nn.Sequential(
            torch.nn.Linear(3 * hidden_channels, hidden_channels),
            torch.nn.LayerNorm(hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(hidden_channels, 1)
        )

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)

        # Process edge features for all edges
        edge_features = self.edge_mlp(edge_attr)

        return x, edge_features

    def decode(self, z, edge_features, edge_label_index):
        src, dst = edge_label_index

        # Handle negative sampling case
        if edge_features.size(0) != edge_label_index.size(1):
            # For negative samples, create dummy edge features
            edge_features = edge_features.mean(dim=0, keepdim=True).repeat(edge_label_index.size(1), 1)

        node_features = torch.cat([
            z[src],
            z[dst],
            edge_features
        ], dim=-1)
        return self.link_predictor(node_features).squeeze(-1)

def train_and_evaluate(model, data, num_epochs=200, k_folds=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    data = data.to(device)

    print(f"Using device: {device}")

    all_results = []
    kf = KFold(n_splits=k_folds, shuffle=True)

    edge_index = data.edge_index.cpu().numpy()
    edge_attr = data.edge_attr.cpu().numpy()
    unique_edges = set()
    edge_to_idx = {}

    for i in range(edge_index.shape[1]):
        edge = tuple(sorted([edge_index[0, i], edge_index[1, i]]))
        if edge not in unique_edges:
            unique_edges.add(edge)
            edge_to_idx[edge] = len(edge_to_idx)

    unique_edges = list(unique_edges)

    for fold, (train_idx, val_idx) in enumerate(kf.split(unique_edges)):
        print(f"\nFold {fold + 1}/{k_folds}")

        train_edges = [unique_edges[i] for i in train_idx]
        val_edges = [unique_edges[i] for i in val_idx]

        # Convert to numpy arrays first
        train_edge_index = np.array([[edge[0], edge[1]] for edge in train_edges]).T
        train_edge_attr = np.array([edge_attr[edge_to_idx[edge]] for edge in train_edges])

        val_edge_index = np.array([[edge[0], edge[1]] for edge in val_edges]).T
        val_edge_attr = np.array([edge_attr[edge_to_idx[edge]] for edge in val_edges])

        # Convert to tensors
        train_edge_index = torch.from_numpy(train_edge_index).to(device)
        train_edge_attr = torch.from_numpy(train_edge_attr).float().to(device)
        val_edge_index = torch.from_numpy(val_edge_index).to(device)
        val_edge_attr = torch.from_numpy(val_edge_attr).float().to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

        best_val_loss = float('inf')
        early_stopping_counter = 0
        train_losses = []
        val_metrics = {'auc': [], 'ap': [], 'loss': []}

        for epoch in range(num_epochs):
            # Training
            model.train()
            optimizer.zero_grad()

            z, edge_features = model(data.x, train_edge_index, train_edge_attr)
            pos_out = model.decode(z, edge_features, train_edge_index)

            # Generate negative samples
            neg_edge_index = negative_sampling(
                train_edge_index,
                num_nodes=data.num_nodes,
                num_neg_samples=train_edge_index.size(1)
            )

            neg_out = model.decode(z, edge_features, neg_edge_index)
            loss = bce_with_logits_loss(pos_out, neg_out)

            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

            # Validation
            model.eval()
            with torch.no_grad():
                z, edge_features = model(data.x, val_edge_index, val_edge_attr)
                pos_out = model.decode(z, edge_features, val_edge_index)

                neg_edge_index = negative_sampling(
                    val_edge_index,
                    num_nodes=data.num_nodes,
                    num_neg_samples=val_edge_index.size(1)
                )

                neg_out = model.decode(z, edge_features, neg_edge_index)
                val_loss = bce_with_logits_loss(pos_out, neg_out)

                # Compute metrics
                pred = torch.cat([pos_out, neg_out]).cpu().numpy()
                true = torch.cat([
                    torch.ones(pos_out.size(0)),
                    torch.zeros(neg_out.size(0))
                ]).numpy()

                auc = roc_auc_score(true, pred)
                ap = average_precision_score(true, pred)

                val_metrics['auc'].append(auc)
                val_metrics['ap'].append(ap)
                val_metrics['loss'].append(val_loss.item())

                if (epoch + 1) % 10 == 0:
                    print(f"Epoch {epoch + 1}: Train Loss = {loss:.4f}, "
                          f"Val Loss = {val_loss:.4f}, AUC = {auc:.4f}, AP = {ap:.4f}")

                scheduler.step(val_loss)

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1

                if early_stopping_counter >= 20:
                    print("Early stopping triggered")
                    break

        fold_results = {
            'fold': fold + 1,
            'train_losses': train_losses,
            'val_metrics': val_metrics,
            'final_auc': auc,
            'final_ap': ap
        }
        all_results.append(fold_results)

    return all_results

def visualize_results(results, network_data, save_path='qkd_results'):
    """Create comprehensive visualizations and analysis"""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    save_path = f"{save_path}_{timestamp}"
    os.makedirs(save_path, exist_ok=True)

    # Find the minimum length across all result arrays
    min_epochs = min(len(result['train_losses']) for result in results)

    # Truncate all arrays to the minimum length
    truncated_results = []
    for result in results:
        truncated_result = {
            'train_losses': result['train_losses'][:min_epochs],
            'val_metrics': {
                'auc': result['val_metrics']['auc'][:min_epochs],
                'ap': result['val_metrics']['ap'][:min_epochs],
                'loss': result['val_metrics']['loss'][:min_epochs]
            },
            'final_auc': result['final_auc'],
            'final_ap': result['final_ap']
        }
        truncated_results.append(truncated_result)

    # Training Metrics
    plt.figure(figsize=(15, 10))

    # Plot training losses
    plt.subplot(2, 2, 1)
    for result in truncated_results:
        plt.plot(result['train_losses'], alpha=0.3)
    mean_train_loss = np.mean([r['train_losses'] for r in truncated_results], axis=0)
    plt.plot(mean_train_loss, 'r-', label='Mean')
    plt.title('Training Loss Evolution')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot validation AUC
    plt.subplot(2, 2, 2)
    for result in truncated_results:
        plt.plot(result['val_metrics']['auc'], alpha=0.3)
    mean_val_auc = np.mean([r['val_metrics']['auc'] for r in truncated_results], axis=0)
    plt.plot(mean_val_auc, 'r-', label='Mean')
    plt.title('Validation AUC Evolution')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.legend()

    # Key Rate vs Distance Analysis
    distances = network_data.edge_attr[:, 2].cpu().numpy()
    key_rates = network_data.edge_attr[:, 0].cpu().numpy()

    plt.subplot(2, 2, 3)
    plt.scatter(distances, key_rates, alpha=0.5)
    plt.xlabel('Distance (km)')
    plt.ylabel('Key Rate (bits/s)')
    plt.yscale('log')
    plt.title('Key Rate vs Distance')

    x_fit = np.linspace(min(distances), max(distances), 100)
    y_fit = np.exp(-0.2 * x_fit)
    plt.plot(x_fit, y_fit * max(key_rates), 'r--', label='Theoretical')
    plt.legend()

    # QBER Distribution
    plt.subplot(2, 2, 4)
    qber_values = network_data.edge_attr[:, 1].cpu().numpy()
    sns.histplot(qber_values, bins=20)
    plt.xlabel('QBER')
    plt.ylabel('Count')
    plt.title('QBER Distribution')

    plt.tight_layout()
    plt.savefig(f"{save_path}/training_metrics.png")
    plt.close()

    # Performance Report
    report = {
        'network_stats': {
            'num_nodes': int(network_data.num_nodes),
            'num_edges': int(len(key_rates)),
            'avg_degree': float(2 * len(key_rates) / network_data.num_nodes),
            'avg_key_rate': float(np.mean(key_rates)),
            'avg_qber': float(np.mean(qber_values)),
            'max_distance': float(np.max(distances))
        },
        'model_performance': {
            'final_metrics': {
                'auc_mean': float(np.mean([r['final_auc'] for r in results])),
                'auc_std': float(np.std([r['final_auc'] for r in results])),
                'ap_mean': float(np.mean([r['final_ap'] for r in results])),
                'ap_std': float(np.std([r['final_ap'] for r in results]))
            },
            'convergence': {
                'final_train_loss_mean': float(np.mean([r['train_losses'][-1] for r in truncated_results])),
                'best_epoch_mean': float(np.mean([np.argmin(r['val_metrics']['loss']) for r in truncated_results]))
            }
        }
    }

    with open(f"{save_path}/performance_report.json", 'w') as f:
        json.dump(report, f, indent=4)

    return report

# Main execution
if __name__ == "__main__":
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)

    # Generate network
    print("Generating QKD network...")
    network = AdvancedQKDNetwork(num_nodes=50)
    data = network.generate_graph_data()

    # Create and train model
    print("Creating and training model...")
    model = AdvancedQKDLinkPredictor(
        in_channels=data.x.size(1),
        edge_attr_channels=data.edge_attr.size(1)
    )

    # Train and evaluate
    results = train_and_evaluate(model, data)

    # Generate visualizations and analysis
    print("Generating visualizations and analysis...")
    save_path = 'qkd_results'
    performance_report = visualize_results(results, data, save_path)

    print(f"\nAnalysis complete. Results saved in: {save_path}")

    # Print summary statistics
    print("\nSummary Statistics:")
    print(f"Number of nodes: {data.num_nodes}")
    print(f"Number of edges: {data.edge_index.size(1) // 2}")
    print(f"Average degree: {data.edge_index.size(1) / data.num_nodes:.2f}")
    print(f"Average key rate: {performance_report['network_stats']['avg_key_rate']:.2e} bits/s")
    print(f"Average QBER: {performance_report['network_stats']['avg_qber']:.3f}")
    print(f"Model AUC: {performance_report['model_performance']['final_metrics']['auc_mean']:.3f} ± "
          f"{performance_report['model_performance']['final_metrics']['auc_std']:.3f}")

Generating QKD network...
Creating and training model...
Using device: cpu

Fold 1/5
Epoch 10: Train Loss = 1.2926, Val Loss = 1.3279, AUC = 0.6575, AP = 0.6474
Epoch 20: Train Loss = 1.0823, Val Loss = 1.1905, AUC = 0.7620, AP = 0.7594
Epoch 30: Train Loss = 0.9176, Val Loss = 1.1153, AUC = 0.7827, AP = 0.7259
Epoch 40: Train Loss = 0.8326, Val Loss = 1.1891, AUC = 0.7637, AP = 0.7288
Epoch 50: Train Loss = 0.7037, Val Loss = 1.1466, AUC = 0.7785, AP = 0.7305
Epoch 60: Train Loss = 0.7188, Val Loss = 1.0975, AUC = 0.7993, AP = 0.7599
Epoch 70: Train Loss = 0.7263, Val Loss = 1.1522, AUC = 0.7759, AP = 0.7398
Early stopping triggered

Fold 2/5
Epoch 10: Train Loss = 0.7056, Val Loss = 1.1737, AUC = 0.7843, AP = 0.7281
Epoch 20: Train Loss = 0.6729, Val Loss = 1.1560, AUC = 0.7887, AP = 0.7383
Early stopping triggered

Fold 3/5
Epoch 10: Train Loss = 0.7089, Val Loss = 1.0001, AUC = 0.8456, AP = 0.8120
Epoch 20: Train Loss = 0.5909, Val Loss = 1.2361, AUC = 0.7967, AP = 0.7647
Epoch 30:

In [ ]:
# import numpy as np
# import torch
# import torch_geometric
# from torch_geometric.data import Data
# from torch_geometric.nn import GATv2Conv, TransformerConv
# from torch_geometric.utils import to_undirected, negative_sampling
# import networkx as nx
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import roc_auc_score, average_precision_score
# from scipy.constants import h, c
# from sklearn.model_selection import KFold
# import json
# from datetime import datetime
# import os
# import torch.nn.functional as F

# def bce_with_logits_loss(pos_out, neg_out):
#     """Custom BCE loss for link prediction"""
#     pos_loss = F.binary_cross_entropy_with_logits(
#         pos_out, torch.ones_like(pos_out)
#     )
#     neg_loss = F.binary_cross_entropy_with_logits(
#         neg_out, torch.zeros_like(neg_out)
#     )
#     return pos_loss + neg_loss

# def to_networkx(data):
#     """Convert PyG data to NetworkX graph"""
#     G = nx.Graph()
#     edge_index = data.edge_index.cpu().numpy()
#     for i in range(edge_index.shape[1]):
#         G.add_edge(edge_index[0, i], edge_index[1, i])
#     return G

# class AdvancedQuantumChannelSimulator:
#     def __init__(self, distance, wavelength=1550e-9, fiber_loss=0.2,
#                  detector_efficiency=0.1, dark_count_rate=1e-6,
#                  atmospheric_visibility=None):
#         self.distance = distance
#         self.wavelength = wavelength
#         self.fiber_loss = fiber_loss
#         self.detector_efficiency = detector_efficiency
#         self.dark_count_rate = dark_count_rate
#         self.atmospheric_visibility = atmospheric_visibility
#         self.photon_energy = h * c / wavelength

#     def calculate_channel_loss(self):
#         fiber_loss_db = self.fiber_loss * self.distance
#         fiber_transmission = 10 ** (-fiber_loss_db/10)

#         if self.atmospheric_visibility:
#             beam_divergence = 1.22 * self.wavelength / 0.1
#             geometric_loss = (0.1 / (beam_divergence * self.distance)) ** 2
#             atmospheric_loss = np.exp(-3.91 * self.distance / self.atmospheric_visibility)
#             total_transmission = fiber_transmission * geometric_loss * atmospheric_loss
#         else:
#             total_transmission = fiber_transmission

#         return total_transmission

#     def simulate_bb84_protocol(self, num_pulses=10000, mean_photon_number=0.1):
#         channel_transmission = self.calculate_channel_loss()
#         received_photons = np.random.poisson(
#             mean_photon_number * channel_transmission * self.detector_efficiency,
#             num_pulses
#         )
#         dark_counts = np.random.poisson(self.dark_count_rate, num_pulses)
#         total_counts = received_photons + dark_counts
#         basis_matches = np.random.choice([0, 1], num_pulses, p=[0.5, 0.5])
#         qber = 0.5 * (1 - np.exp(-2 * self.distance / 100))
#         errors = np.random.choice([0, 1], num_pulses, p=[1-qber, qber])
#         matched_pulses = total_counts * basis_matches
#         raw_key_rate = np.sum(matched_pulses) / num_pulses
#         final_key_rate = raw_key_rate * (1 - 2 * h2(qber))

#         return {
#             'qber': qber,
#             'raw_key_rate': raw_key_rate,
#             'final_key_rate': final_key_rate,
#             'channel_loss_db': -10 * np.log10(channel_transmission),
#             'dark_count_probability': np.mean(dark_counts > 0)
#         }

# def h2(x):
#     """Binary entropy function"""
#     return -x * np.log2(x) - (1-x) * np.log2(1-x) if 0 < x < 1 else 0

# class AdvancedQKDNetwork:
#     def __init__(self, num_nodes=20):
#         self.num_nodes = num_nodes
#         self.positions = self._generate_realistic_topology()

#     def _generate_realistic_topology(self):
#         centers = np.random.multivariate_normal(
#             mean=[0, 0],
#             cov=[[100, 0], [0, 100]],
#             size=3
#         )

#         positions = []
#         for _ in range(self.num_nodes):
#             center = centers[np.random.randint(0, 3)]
#             pos = center + np.random.multivariate_normal(
#                 mean=[0, 0],
#                 cov=[[10, 0], [0, 10]]
#             )
#             positions.append(pos)

#         return np.array(positions)

#     def generate_graph_data(self):
#         distances = np.zeros((self.num_nodes, self.num_nodes))
#         for i in range(self.num_nodes):
#             for j in range(i + 1, self.num_nodes):
#                 distances[i, j] = distances[j, i] = np.linalg.norm(
#                     self.positions[i] - self.positions[j]
#                 )

#         edges = []
#         edge_attrs = []

#         for i in range(self.num_nodes):
#             for j in range(i + 1, self.num_nodes):
#                 if distances[i, j] < 100:
#                     simulator = AdvancedQuantumChannelSimulator(
#                         distance=distances[i, j],
#                         atmospheric_visibility=20000 if np.random.random() < 0.2 else None
#                     )
#                     results = simulator.simulate_bb84_protocol()

#                     if results['final_key_rate'] > 0:
#                         edges.append([i, j])
#                         edge_attrs.append([
#                             results['final_key_rate'],
#                             results['qber'],
#                             distances[i, j],
#                             results['channel_loss_db'],
#                             results['dark_count_probability']
#                         ])

#         edge_index = torch.tensor(edges).t().contiguous()
#         edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

#         G = nx.Graph()
#         G.add_edges_from(edges)

#         node_features = []
#         for i in range(self.num_nodes):
#             features = [
#                 self.positions[i, 0],
#                 self.positions[i, 1],
#                 G.degree(i) if i in G else 0,
#                 nx.betweenness_centrality(G).get(i, 0) if i in G else 0
#             ]
#             node_features.append(features)

#         return Data(
#             x=torch.tensor(node_features, dtype=torch.float),
#             edge_index=to_undirected(edge_index),
#             edge_attr=edge_attr,
#             pos=torch.tensor(self.positions, dtype=torch.float)
#         )

# # Main execution
# if __name__ == "__main__":
#     # Set random seeds for reproducibility
#     torch.manual_seed(42)
#     np.random.seed(42)

#     # Generate network
#     print("Generating QKD network...")
#     network = AdvancedQKDNetwork(num_nodes=20)
#     data = network.generate_graph_data()

#     # Create output directory
#     save_path = 'qkd_results'
#     os.makedirs(save_path, exist_ok=True)

#     # Visualize network topology
#     plt.figure(figsize=(10, 8))
#     pos = {i: data.pos[i].numpy() for i in range(data.num_nodes)}
#     G = to_networkx(data)
#     nx.draw(G, pos, node_color='lightblue',
#             with_labels=True, node_size=500,
#             font_size=10, font_weight='bold')
#     plt.savefig(f"{save_path}/network_topology.png")
#     plt.close()

#     print(f"Analysis complete. Results saved in: {save_path}")

#     # Print basic network statistics
#     print("\nNetwork Statistics:")
#     print(f"Number of nodes: {data.num_nodes}")
#     print(f"Number of edges: {data.edge_index.size(1) // 2}")  # Divide by 2 for undirected
#     print(f"Average degree: {data.edge_index.size(1) / data.num_nodes:.2f}")